In [1]:
import weaviate
client = weaviate.connect_to_local()
client.is_ready()

True

In [2]:
from weaviate.classes.config import Configure

client.collections.delete("CollectionWithAutoMTEnabled")
multi_collection = client.collections.create(
    name="CollectionWithAutoMTEnabled",
    # Enable automatic tenant creation
    multi_tenancy_config=Configure.multi_tenancy(
        enabled=True, auto_tenant_creation=True
    ),
)

In [3]:
# lets import data in batch to newly created tenants

data = [
    {"tenant": "tenant1", "text": "Text for tenant1"},
    {"tenant": "tenant2", "text": "Text for tenant2"},
]

with client.batch.fixed_size(batch_size=200) as batch:
    for obj in data:
        properties = {
            "text": obj["text"],
        }
        batch.add_object(
            collection="CollectionWithAutoMTEnabled",
            properties=properties,
            tenant=obj["tenant"],
        )

In [4]:
tenant1 = client.collections.get("CollectionWithAutoMTEnabled").with_tenant("tenant1")
tenant1.query.fetch_objects().objects[0].properties

{'text': 'Text for tenant1'}

In [5]:
tenant1 = client.collections.get("CollectionWithAutoMTEnabled").with_tenant("tenant2")
tenant1.query.fetch_objects().objects[0].properties

{'text': 'Text for tenant2'}

In [6]:
client.collections.get("CollectionWithAutoMTEnabled").tenants.get()

{'tenant1': Tenant(name='tenant1', activityStatus=<TenantActivityStatus.HOT: 'HOT'>),
 'tenant2': Tenant(name='tenant2', activityStatus=<TenantActivityStatus.HOT: 'HOT'>)}